In [ ]:
import os
# /content/gdrive/MyDrive/ISBI timepoints
entries = os.listdir('D:/ISBI original')
print(entries)
print(len(entries))

In [ ]:
from tqdm import tqdm
import os
import time
from random import randint

!pip install albumentations==0.4.6

import numpy as np
from scipy import stats
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold

import nibabel as nib
!pip install pydicom
import pydicom as pdm
!pip install nilearn
import nilearn as nl
import nilearn.plotting as nlplt
import h5py

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import seaborn as sns
import imageio
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif
from IPython.display import clear_output
from IPython.display import YouTubeVideo

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn import MSELoss

!pip install albumentations==0.4.6
import albumentations as A
from albumentations import Compose, HorizontalFlip
from albumentations.pytorch import ToTensor, ToTensorV2 

import warnings
warnings.simplefilter("ignore")

In [ ]:
!pip install torchio==0.18.33 --quiet
!pip install pandas --quiet
!pip install matplotlib --quiet
!pip install seaborn --quiet
!pip install scikit-image --quiet
!curl -s -o colormap.txt https://raw.githubusercontent.com/thenineteen/Semiology-Visualisation-Tool/master/slicer/Resources/Color/BrainAnatomyLabelsV3_0.txt
!curl -s -o slice_7t.jpg https://static.healthcare.siemens.com/siemens_hwem-hwem_ssxa_websites-context-root/wcm/idc/groups/public/@global/@imaging/@mri/documents/image/mda5/nje2/~edisp/siemens-healthineers_mri_magnetom-terra_3-07093947/~renditions/siemens-healthineers_mri_magnetom-terra_3-07093947~8.jpg
!curl -s -o slice_histo.jpg https://bcf.technion.ac.il/wp-content/uploads/2018/05/Neta-histology-slice-626.jpg
!curl -s -o vhp.zip ftp://lhcftp.nlm.nih.gov/Open-Access-Datasets/Visible-Human/Sample-Data/Six%20slices%20from%20the%20Visible%20Male.zip
!unzip -o vhp.zip > /dev/null

In [ ]:
import copy
import pprint
import torch
!pip install torchio
import torchio as tio
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import torchvision.transforms as transforms
from torchvision.utils import save_image
sns.set_style("whitegrid", {'axes.grid' : False})
%config InlineBackend.figure_format = 'retina'
torch.manual_seed(14041931)
print('TorchIO version:', tio.__version__)

In [ ]:


# generate random integer values
from random import seed
from random import randint



In [ ]:
import os
# /content/gdrive/MyDrive/ISBI timepoints
entries = os.listdir('D:/ISBI original')
print(entries)
print(len(entries))

In [ ]:
i=0
while(i<20):
    images = []
    data_types = ['_flair_pp.nii', '_T1WKS_pp.nii','_T1W_pp.nii','_T2W_pp.nii']
    folder_path='D:/ISBI original'
    # x=entries[i]+"/preprocessed/"+entries[i]
    m=entries[i]+"/"+entries[i]
    img_path = os.path.join(folder_path, m)
    msk_path = os.path.join(folder_path, m)
    print(img_path)
    print(msk_path)


    flair=img_path+'_flair_pp.nii'
    t1wks=img_path+'_mprage_pp.nii'
    t1w=img_path+'_pd_pp.nii'
    t2w=img_path+'_t2_pp.nii'
    mask=msk_path+'_mask1.nii'

    flair_img=tio.ScalarImage(flair)
    t1wks_img=tio.ScalarImage(t1wks)
    t1w_img=tio.ScalarImage(t1w)
    t2w_img=tio.ScalarImage(t2w)
    mask_img=tio.LabelMap(mask)

    img= tio.Subject(f=flair_img,
                     t1wk=t1wks_img,
                     t1=t1w_img,
                     t2=t2w_img,
                     m=mask_img)
    
    # random_flip = tio.RandomFlip(axes=['inferior-superior'], flip_probability=0.6)
    # img = random_flip(img)

    random_affine=tio.RandomAffine(degrees=20,image_interpolation='bspline')
    img=random_affine(img)

    # random_flip = tio.RandomFlip(axes=['inferior-superior'], flip_probability=1)
    # img = random_flip(img)

    max_displacement = 10, 10, 0  # in x, y and z directions
    random_elastic = tio.RandomElasticDeformation(max_displacement=max_displacement,image_interpolation='bspline')
    img=random_elastic(img)


    sample_flair=img.f.data.numpy().squeeze()
    sample_t1wks=img.t1wk.data.numpy().squeeze()
    sample_t1w=img.t1.data.numpy().squeeze()
    sample_t2w=img.t2.data.numpy().squeeze()
    sample_mask=img.m.data.numpy().squeeze()
    # sample_mask=img[4]
    value = randint(0, 1)
    print(value)
    if(value==1):
        sample_mask=np.flip(sample_mask,axis=1)
        sample_flair=np.flip(sample_flair,axis=1)
        sample_t1w=np.flip(sample_t1w,axis=1)
        sample_t1wks=np.flip(sample_t1wks,axis=1)
        sample_t2w=np.flip(sample_t2w,axis=1)

    sample_flair=nib.Nifti1Image(sample_flair,np.eye(4))
    # print(sample_flair.get_data_dtype() == np.dtype(np.int16))
    sample_t1wks=nib.Nifti1Image(sample_t1wks,np.eye(4))
    sample_t1w=nib.Nifti1Image(sample_t1w,np.eye(4))
    sample_t2w=nib.Nifti1Image(sample_t2w,np.eye(4))
    sample_mask=nib.Nifti1Image(sample_mask,np.eye(4))
    # print("sample_flair", sample_flair.type)

    temp=entries[i]
    new_num=int(temp[-2:])+20


# /content/gdrive/MyDrive/ISBI timepoints/p1 diff 2-1/training_01_flair_pp.nii
    folder_path='D:/ISBI original aug'
    x='MS_Training_'+str(new_num)
    img_path = os.path.join(folder_path, x)
    img_path_type = img_path+'/'+x+'_flair_pp.nii'
    print(img_path_type)
    nib.save(sample_flair,img_path_type)



    folder_path='D:/ISBI original aug'
    x='MS_Training_'+str(new_num)
    img_path = os.path.join(folder_path, x)
    img_path_type = img_path+'/'+x+'_mprage_pp.nii'
    print(img_path_type)
    nib.save(sample_t1wks,img_path_type)



    folder_path='D:/ISBI original aug'
    x='MS_Training_'+str(new_num)
    img_path = os.path.join(folder_path, x)
    img_path_type = img_path+'/'+x+'_pd_pp.nii'
    print(img_path_type)
    nib.save(sample_t1w,img_path_type)



    folder_path='D:/ISBI original aug'
    x='MS_Training_'+str(new_num)
    img_path = os.path.join(folder_path, x)
    img_path_type = img_path+'/'+x+'_t2_pp.nii'
    print(img_path_type)
    nib.save(sample_t2w,img_path_type)



    folder_path='D:/ISBI original aug'
    x='MS_Training_'+str(new_num)
    img_path = os.path.join(folder_path, x)
    img_path_type = img_path+'/'+x+'_mask1.nii'
    print(img_path_type)
    nib.save(sample_mask,img_path_type)

    i+=1
    


